In [1]:
import tkinter as tk
from tkinter import simpledialog, messagebox
from PIL import Image, ImageTk
import pandas as pd
import difflib
from langdetect import detect
from langdetect import DetectorFactory
from googletrans import Translator
import mysql.connector
DetectorFactory.seed = 0
chatbot_data = pd.read_csv('chatbot_rules.csv')
chat_history = []
def get_response(user_message):
    user_language = detect(user_message)
    user_message = user_message.lower()
    target_language = 'ta' if user_language == 'ta' else 'en'
    user_message_translated = translate_message(user_message, target_language)

    best_match = ""
    highest_similarity = 0.0

    for index, row in chatbot_data.iterrows():
        question = row['Question'].lower()
        question_language = detect(question)
        question_translated = translate_message(question, target_language)

        similarity = similar(user_message_translated, question_translated)
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = row['Response']

    response = translate_message(best_match, user_language)
    chat_history.append(("You", user_message, "Chatbot", response))
    return response
def similar(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()
def translate_message(message, target_language):
    translator = Translator()
    translation = translator.translate(message, dest=target_language)
    return translation.text
def send_message(user_input):
    chat_text.config(state=tk.NORMAL)
    chat_text.insert(tk.END, "நீங்கள்: " + user_input + "\n")
    response = get_response(user_input)
    chat_text.insert(tk.END, "மெய்நிகர் நண்பர்: " + response + "\n")
    chat_text.config(state=tk.DISABLED)
    user_entry.delete(0, tk.END)
def display_suggestion_response(suggestion):
    user_entry.delete(0, tk.END) 
    user_entry.insert(tk.END, suggestion) 
    send_message(suggestion)
def update_suggestions_and_type_prompt(event):
    user_input = user_entry.get()
    user_language = detect(user_input)
    
    suggestions = []
    if user_language not in ['ta', 'en']:
        for question in chatbot_data['Question']:
            translated_question = translate_message(question, user_language)
            if user_input.lower() in translated_question.lower():
                suggestions.append(translated_question)
    else:
        suggestions = [question for question in chatbot_data['Question'] if user_input in question.lower()]
    for frame in suggestion_frames:
        frame.destroy()
    row_count = 0
    col_count = 0
    for suggestion in suggestions:
        frame = tk.Frame(suggestion_frame, bg="white", relief=tk.SOLID, borderwidth=1)
        frame.grid(row=row_count, column=col_count, pady=5, padx=5)
        suggestion_list = tk.Listbox(frame, selectmode=tk.SINGLE, width=30, height=1, bg="white", relief=tk.SOLID, borderwidth=0)
        suggestion_list.pack(side=tk.LEFT)
        suggestion_list.insert(tk.END, suggestion)
        suggestion_list.bind("<ButtonRelease-1>", lambda event, s=suggestion: display_suggestion_response(s))
        suggestion_frames.append(frame)
        col_count += 1
        if col_count > 1:
            col_count = 0
            row_count += 1

def save_to_excel():
    df = pd.DataFrame(chat_history, columns=["User", "User Message", "Chatbot", "Chatbot Response"])
    df.to_excel("chat_history.xlsx", index=False)
def handle_feedback():
    feedback_form = tk.Toplevel(root)
    feedback_form.title("Feedback Form")

    tk.Label(feedback_form, text="Name:").grid(row=0, column=0, padx=10, pady=5)
    name_entry = tk.Entry(feedback_form)
    name_entry.grid(row=0, column=1, padx=10, pady=5)

    tk.Label(feedback_form, text="Phone Number:").grid(row=1, column=0, padx=10, pady=5)
    phone_entry = tk.Entry(feedback_form)
    phone_entry.grid(row=1, column=1, padx=10, pady=5)

    tk.Label(feedback_form, text="Email:").grid(row=2, column=0, padx=10, pady=5)
    email_entry = tk.Entry(feedback_form)
    email_entry.grid(row=2, column=1, padx=10, pady=5)

    tk.Label(feedback_form, text="Feedback:").grid(row=3, column=0, padx=10, pady=5)
    feedback_entry = tk.Entry(feedback_form)
    feedback_entry.grid(row=3, column=1, padx=10, pady=5)

    def submit_feedback():
        feedback_data = {
            "Name": name_entry.get(),
            "Phone Number": phone_entry.get(),
            "Email": email_entry.get(),
            "Feedback": feedback_entry.get()
        }
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Ashi0318",
            database="temple_chatbot"
        )
        cursor = conn.cursor()
        cursor.execute("INSERT INTO feedbacks (name, phone, email, feedback) VALUES (%s, %s, %s, %s)",
                       (feedback_data["Name"], feedback_data["Phone Number"], feedback_data["Email"], feedback_data["Feedback"]))
        conn.commit()
        conn.close()

        messagebox.showinfo("Feedback Submitted", "Thank you for your valuable feedback!")
        feedback_form.destroy()

    submit_button = tk.Button(feedback_form, text="Submit", command=submit_feedback, bg="green", fg="white")
    submit_button.grid(row=4, column=0, columnspan=2, pady=10)
root = tk.Tk()
root.title("கோவிலின் சாட்போட்")
background_image = Image.open("gopuram.jpg")
background_image = ImageTk.PhotoImage(background_image)
background_label = tk.Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)
padding = 45
chat_text = tk.Text(root, height=15, width=60, bg="lightyellow")
chat_text.pack(pady=padding)
chat_text.config(state=tk.DISABLED)
suggestion_frame = tk.Frame(root)
suggestion_frame.pack(pady=padding)
suggestion_frames = []
search_frame = tk.Frame(root)
search_frame.pack(pady=padding)
user_entry = tk.Entry(search_frame, width=60)
user_entry.pack(side=tk.LEFT)
send_button = tk.Button(search_frame, text="அனுப்பு", command=lambda: send_message(user_entry.get()), bg="green", fg="white")
send_button.pack(side=tk.RIGHT)
save_to_excel_button = tk.Button(root, text="எக்செல் இல் சேமிக்கவும்", command=save_to_excel, bg="blue", fg="white")
save_to_excel_button.pack(pady=padding)
feedback_button = tk.Button(root, text="பின்னூட்டம்", command=handle_feedback, bg="orange", fg="white")
feedback_button.pack(pady=padding)
user_entry.config(bg="white")
user_entry.bind("<KeyRelease>", update_suggestions_and_type_prompt)
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\suhai\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\suhai\AppData\Local\Temp\ipykernel_22204\788842346.py", line 54, in update_suggestions_and_type_prompt
    user_language = detect(user_input)
                    ^^^^^^^^^^^^^^^^^^
  File "C:\Users\suhai\anaconda3\Lib\site-packages\langdetect\detector_factory.py", line 130, in detect
    return detector.detect()
           ^^^^^^^^^^^^^^^^^
  File "C:\Users\suhai\anaconda3\Lib\site-packages\langdetect\detector.py", line 136, in detect
    probabilities = self.get_probabilities()
                    ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\suhai\anaconda3\Lib\site-packages\langdetect\detector.py", line 143, in get_probabilities
    self._detect_block()
  File "C:\Users\suhai\anaconda3\Lib\site-packages\langdetect\detector.py", line 150, in _detect_block
    raise LangDe